Copyright (c) 2012-2022 Esri R&D Center Zurich

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

  https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
A copy of the license is available in the repository's LICENSE file.

# PyPRT - Building Modeling Optimization

In this notebook, we are using a CGA rule that allows us to generate modern and sustainable buildings. The architecture of these buildings is focused on the green aspect of the building. Therefore, in addition to be modern and esthetical, the generated building answers also to the actual requirements of the city which are to encourage the integration of greenery on the buildings.

![Bosco Verticale in Milan](images/bosco_verticale_milan.png)

Bosco Verticale in Milan

#### Imports

In [ ]:
import sys
import os
from scipy import optimize
import numpy as np
import pyvista
import trame
from vtkmodules.vtkIOImport import vtkGLTFImporter

In [ ]:
CWD = os.getcwd() # workaround: PyPRT initialization unexpectedly changes current working directory
DATA = os.path.join(CWD, 'data')

In [ ]:
import pyprt
pyprt.initialize_prt()

if(not pyprt.is_prt_initialized()):
    raise Exception("PRT is not initialized")

## Optimization Problem Definition

Given the architectural design of this building (defined by the CGA rule) and the parcel, which value of the **lot coverage**, **tier height** and **building shape** should we choose in order to maximize the building green potential (green area).

![Building Parameters](images/green_building_parameters.png)

In this example, only the previously mentioned variables were taken into account for simplicity purposes. For instance, the building parcel geographical orientation is not known. Therefore, the sun direction could not be derived. The well-being of the plants and greenery is thus not considered.

### Given: building architecture (CGA Rule Package) and parcel to redevelop

In [ ]:
rpk_green = os.path.join(DATA, 'envelope2002.rpk')
attrs_green = {'report_but_not_display_green': True, 'seed': 666}
shape_geo = pyprt.InitialShape(os.path.join(DATA, 'building_parcel.obj'))

### Greenery Optimization

Step 1 : optimize the total green area reported (including the balconies trees). This will be optimized taking into account several parameters:
- the shape of the building (1 = orthogonal, 2 = L shape, 3 = U shape, 4 = O shapes)
- the lot coverage
- the height of the first tier

#### Optimization Variables

In [ ]:
goal_str_green = 'Greenery Area' + '_sum'

input_attr_green = ['lot_coverage_parameter',
                    'height_first_tier', 'shape_of_building']
x0_green = [float(50), float(9), float(round(2))]

In [ ]:
m = pyprt.ModelGenerator([shape_geo])

In [ ]:
def fct_green(x):

    attrs_green[input_attr_green[0]] = x[0]
    attrs_green[input_attr_green[1]] = x[1]
    attrs_green[input_attr_green[2]] = float(round(x[2]))
    generated_mod_green = m.generate_model([attrs_green], rpk_green,
                          'com.esri.pyprt.PyEncoder', {'emitGeometry': False})

    if generated_mod_green:
        rep_green = generated_mod_green[0].get_report()
        return -rep_green[goal_str_green]
    else:
        print('Error in optimization process.')

#### Variables boundaries

In [ ]:
bounds_green = [(0, 100), (8, 13), (1, 4)]

The SciPy optimization library is extensively used within Python community. However, **any** optimization algorithm works with PyPRT. In this notebook, the optimization problem to solve is complex and highly non linear. The differential evolution algorithm from SciPy is particularly suited to our problem because it is stochastic and, thus, does not need any gradient computation to find the minimum. The choice of optimization algorithm depends on the problem and, in our case, the CGA rule.

In [ ]:
res_green = optimize.differential_evolution(
    fct_green, bounds_green, tol=0.15, seed=666)
res_green

#### Results Visualization

In [ ]:
export_file_name = 'ex7_optimalmodel1'
enc_options = {'baseName': export_file_name,
               'outputPath': os.path.join(CWD, 'ex7_output')}
os.makedirs(enc_options['outputPath'], exist_ok=True)

attrs_green[input_attr_green[0]] = res_green.x[0]
attrs_green[input_attr_green[1]] = res_green.x[1]
attrs_green[input_attr_green[2]] = float(round(res_green.x[2]))
green_model = m.generate_model(
    [attrs_green], rpk_green, 'com.esri.prt.codecs.GLTFEncoder', enc_options)

In [ ]:
importer = vtkGLTFImporter()
importer.SetFileName(os.path.join(
    enc_options['outputPath'], export_file_name+'_0.glb'))
pl = pyvista.Plotter()
pl.set_background('white')
pl.show_axes()
pl.enable_depth_peeling()
pl.add_light(pyvista.Light(light_type='scene light'))
importer.SetRenderWindow(pl.ren_win)
importer.Update()
pl.show(cpos='xy')

## Multi-Objectives Optimization

Step 2 : optimize the green area reported including the balconies trees AND the floor area in order to avoid thin walls. We will keep O-shape building. This will be optimized taking into account several parameters:

- the lot coverage
- the height of the first tier

#### Optimization Variables

In [ ]:
goal_str_green = 'Greenery Area' + '_sum'
goal_str_floor = 'Floor area' + '_sum'
# this allows having similar range between the green area and the floor area
floor_weight = 0.07

input_attr_green = ['lot_coverage_parameter', 'height_first_tier']
x0_green = [float(50), float(9)]

In [ ]:
attrs_green = {'report_but_not_display_green': True, 'seed': 666, 'shape_of_building': 4.0}

In [ ]:
def fct_green(x):

    attrs_green[input_attr_green[0]] = x[0]
    attrs_green[input_attr_green[1]] = x[1]
    generated_mod_green = m.generate_model([attrs_green], rpk_green,
                          'com.esri.pyprt.PyEncoder', {'emitGeometry': False})

    if generated_mod_green:
        rep_green = generated_mod_green[0].get_report()
        return -(rep_green[goal_str_green]+floor_weight*rep_green[goal_str_floor])
    else:
        print('Error in optimization process.')

In [ ]:
bounds_green = [(0, 100), (8, 13)]

In [ ]:
res_green = optimize.differential_evolution(
    fct_green, bounds_green, tol=0.1, seed=666)
res_green

#### Results Visualization

In [ ]:
export_file_name = 'ex7_optimalmodel2'
enc_options['baseName'] = export_file_name

attrs_green[input_attr_green[0]] = res_green.x[0]
attrs_green[input_attr_green[1]] = res_green.x[1]
green_model = m.generate_model(
    [attrs_green], rpk_green, 'com.esri.prt.codecs.GLTFEncoder', enc_options)

In [ ]:
importer = vtkGLTFImporter()
importer.SetFileName(os.path.join(
    enc_options['outputPath'], export_file_name+'_0.glb'))
pl = pyvista.Plotter()
pl.set_background('white')
pl.show_axes()
pl.enable_depth_peeling()
pl.add_light(pyvista.Light(light_type='scene light'))
importer.SetRenderWindow(pl.ren_win)
importer.Update()
pl.show(cpos='xy')

In [ ]:
pyprt.shutdown_prt()